# Snowflake Cortex Cost Monitoring and Analysis

This notebook provides a robust set of examples for monitoring and analyzing costs associated with Snowflake Cortex LLM functions and Cortex Analyst. The examples use Snowflake's `ACCOUNT_USAGE` views and system tables to help administrators and analysts understand where credits are being consumed.

**Prerequisites:**

- You must have the **ACCOUNTADMIN** role or a role with access to the `SNOWFLAKE` database and `ACCOUNT_USAGE` schema.
- The `SNOWFLAKE` database should be visible in your session.
- For analyst-specific monitoring, ensure that you have enabled and used Cortex Analyst in your account.

Each section below contains SQL queries you can run directly in a Snowflake worksheet or notebook. Feel free to adjust date ranges (via `DATEADD`) and filters to suit your environment.

## 1. Daily AI Services Consumption

Use the `METERING_DAILY_HISTORY` view to see how many credits have been consumed each day by AI Services (which includes Cortex LLM function calls, Document AI, Cortex Search, etc.). Adjust the date range as needed.

The `SERVICE_TYPE` column contains values like `AI_SERVICES`, `WAREHOUSE_METERING`, `STORAGE`, etc. We're interested in AI services.

In [ ]:
-- Daily credit consumption for AI Services over the last 30 days
SELECT
    USAGE_DATE,
    SUM(CREDITS_USED) AS credits_used
FROM SNOWFLAKE.ACCOUNT_USAGE.METERING_DAILY_HISTORY
WHERE SERVICE_TYPE = 'AI_SERVICES'
  AND USAGE_DATE >= DATEADD(day, -30, CURRENT_DATE())
GROUP BY USAGE_DATE
ORDER BY USAGE_DATE;

## 2. Cortex Functions Usage Overview

The `CORTEX_FUNCTIONS_USAGE_HISTORY` view provides aggregated token and credit usage for each function and model in one‑hour windows. The following query summarizes total tokens and credits for each function and model over the past 7 days.

This is useful for understanding which functions or models are consuming the most credits. If you see unexpected spikes, drill down using the query-level view in the next section.

In [ ]:
-- Summarize token and credit usage by function and model over the last 7 days
SELECT
    FUNCTION_NAME,
    MODEL_NAME,
    SUM(TOKENS)         AS total_tokens,
    SUM(TOKEN_CREDITS)  AS total_credits
FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_FUNCTIONS_USAGE_HISTORY
WHERE START_TIME >= DATEADD(day, -7, CURRENT_TIMESTAMP())
GROUP BY 1, 2
ORDER BY total_credits DESC;

## 3. Per-Query Cortex Function Usage

To identify individual queries that are consuming significant credits, use the `CORTEX_FUNCTIONS_QUERY_USAGE_HISTORY` view. Each row corresponds to a `QUERY_ID`, `FUNCTION_NAME` and `MODEL_NAME`, showing the number of tokens and credits consumed by that query.

The example below lists the top 10 queries by credit consumption in the last 7 days. You can join this to `QUERY_HISTORY` to get the text of the query or user information.

In [ ]:
-- Top 10 queries by AI token credit consumption in the last 7 days
SELECT
    q.query_id,
    c.model_name,
    c.function_name,
    c.tokens,
    c.token_credits,
    q.user_name
FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_FUNCTIONS_QUERY_USAGE_HISTORY c
JOIN SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY q
  ON c.query_id = q.query_id
WHERE q.start_time >= DATEADD(day, -7, CURRENT_TIMESTAMP())
ORDER BY c.token_credits DESC
LIMIT 10;

## 4. Top Models and Users

You can further analyze usage by model or by user. The queries below compute total credits by model and by user over the past 30 days. Replace the date range as needed.

In [ ]:
-- Top models by total credits over the last 30 days
SELECT
    MODEL_NAME,
    SUM(TOKEN_CREDITS) AS total_credits
FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_FUNCTIONS_USAGE_HISTORY
WHERE START_TIME >= DATEADD(day, -30, CURRENT_TIMESTAMP())
GROUP BY MODEL_NAME
ORDER BY total_credits DESC
LIMIT 10;

In [ ]:
-- Top users by AI Services credits consumed over the last 30 days
SELECT
    q.user_name,
    SUM(c.token_credits) AS total_credits
FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_FUNCTIONS_QUERY_USAGE_HISTORY c
JOIN SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY q
  ON c.query_id = q.query_id
WHERE q.start_time >= DATEADD(day, -30, CURRENT_TIMESTAMP())
GROUP BY q.user_name
ORDER BY total_credits DESC
LIMIT 10;

## 5. Cortex Analyst Usage

Cortex Analyst is a text-to-SQL service whose usage can be tracked using the `CORTEX_ANALYST_USAGE_HISTORY` view. Each row aggregates requests, credits, and user information over one-hour windows.

The queries below show overall analyst consumption and break it down by user. Adjust the date range to meet your needs.

In [ ]:
-- Total Cortex Analyst requests and credits over the last 30 days
SELECT
    DATE_TRUNC('day', START_TIME) AS usage_day,
    SUM(REQUEST_COUNT) AS total_requests,
    SUM(CREDITS)        AS total_credits
FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_ANALYST_USAGE_HISTORY
WHERE START_TIME >= DATEADD(day, -30, CURRENT_TIMESTAMP())
GROUP BY usage_day
ORDER BY usage_day;

In [ ]:
-- Cortex Analyst usage by user over the last 30 days
SELECT
    USERNAME,
    SUM(REQUEST_COUNT) AS total_requests,
    SUM(CREDITS)       AS total_credits,
    SUM(CREDITS) / NULLIF(SUM(REQUEST_COUNT), 0) AS credits_per_request
FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_ANALYST_USAGE_HISTORY
WHERE START_TIME >= DATEADD(day, -30, CURRENT_TIMESTAMP())
GROUP BY USERNAME
ORDER BY total_credits DESC
LIMIT 10;

## 6. Resource Monitors and Budgets

Resource monitors and budgets help prevent unexpected charges.

- **Resource monitors** can suspend warehouses at defined thresholds. Use `SHOW RESOURCE MONITORS` to view existing monitors and their quotas.
- **Budgets** provide monthly spend targets and send notifications when spending is projected to exceed the limit. Use `SHOW SNOWFLAKE.CORE.BUDGET` to list budgets you have access to.

The queries below return existing monitors and budgets.

In [ ]:
-- List all resource monitors and their status
SHOW RESOURCE MONITORS;

In [ ]:
-- List all budgets (requires appropriate privileges)
SHOW SNOWFLAKE.CORE.BUDGET;

## 8. Additional Views for AI Services

Snowflake exposes several other views for specialized services. Depending on which services you use, you can run the following queries:

- **Document AI:** `SNOWFLAKE.ACCOUNT_USAGE.DOCUMENT_AI_USAGE_HISTORY` provides credit usage by document AI operations.
- **Cortex Search:** `SNOWFLAKE.ACCOUNT_USAGE.CORTEX_SEARCH_DAILY_USAGE_HISTORY` and `CORTEX_SEARCH_SERVING_USAGE_HISTORY` show credits consumed for token embedding and serving compute.
- **Cortex Document Processing:** `SNOWFLAKE.ACCOUNT_USAGE.CORTEX_DOCUMENT_PROCESSING_USAGE_HISTORY` shows credits consumed for parsing documents with `AI_PARSE_DOCUMENT`.

Below are sample queries for these views.

In [ ]:
-- Document AI usage over the last 30 days
SELECT
    DATE_TRUNC('day', START_TIME) AS usage_day,
    SUM(CREDITS_USED) AS total_credits,
    SUM(PAGE_COUNT) AS total_pages,
    SUM(DOCUMENT_COUNT) AS total_documents
FROM SNOWFLAKE.ACCOUNT_USAGE.DOCUMENT_AI_USAGE_HISTORY
WHERE START_TIME >= DATEADD(day, -365, CURRENT_TIMESTAMP())
GROUP BY usage_day
ORDER BY usage_day;

In [ ]:
-- Cortex Search embedding and serving usage over the last 30 days
SELECT
    SERVICE_NAME,
    DATE_TRUNC('day', USAGE_DATE) AS usage_day,
    CONSUMPTION_TYPE,
    SUM(CREDITS)    AS embed_credits
FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_SEARCH_DAILY_USAGE_HISTORY
WHERE USAGE_DATE >= DATEADD(day, -30, CURRENT_TIMESTAMP())
GROUP BY SERVICE_NAME, usage_day, CONSUMPTION_TYPE
ORDER BY usage_day;

## 9. Token Count

You can count the input tokens to understand what the token utilization would be if you were to run on of the scalar functions.

Below are sample queries for these views.

In [ ]:
-- Count tokens for a single prompt with a specific model
SELECT SNOWFLAKE.CORTEX.COUNT_TOKENS('llama3.1-70b', 'what is a large language model?') AS tokens;

In [ ]:
-- For each row, count tokens in a text column
SELECT
  id,
  SNOWFLAKE.CORTEX.COUNT_TOKENS('llama3.1-70b', content) AS prompt_tokens
FROM my_db.my_schema.my_table
ORDER BY prompt_tokens DESC;